In [89]:
from typing import Union
from langchain.llms import LlamaCpp, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents import AgentExecutor
import config
import os

## **Load the LLM**

In [90]:
os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

In [91]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## **Define Tools**

In [92]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

get_word_length.invoke("abc")

3

In [93]:
tools = [get_word_length]

## **Create Promp**

In [94]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## **Bind tools to LLM**

In [95]:
llm_with_tools = llm.bind_functions(tools)

## **Crate the Agent**

In [96]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [97]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [98]:
list(agent_executor.stream({"input": "How many letters in the word hello"}))



> Entering new AgentExecutor chain...




> Finished chain.


[{'output': '', 'messages': [AIMessage(content='')]}]

In [99]:
llm.invoke("How many letters in the word casa")

AIMessage(content='There are 4 letters in the word "casa".', response_metadata={'finish_reason': 'stop', 'logprobs': None})

## **Adding memory**

In [100]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [101]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [102]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [105]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?",
                      "chat_history": chat_history})



> Entering new AgentExecutor chain...


> Finished chain.


> Entering new AgentExecutor chain...
I apologize for the mistake. "Educa" is not a real word. Let me calculate the length of a valid word for you.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content=''),
  HumanMessage(content='how many letters in the word hello?'),
  AIMessage(content=''),
  HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='')],
 'output': 'I apologize for the mistake. "Educa" is not a real word. Let me calculate the length of a valid word for you.'}